### Import Google Sentiment stuff

In [1]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

### Others imports

In [ ]:
client = language.LanguageServiceClient()

In [2]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import sys,csv
import matplotlib
from tqdm import tqdm

In [3]:
db = sql.connect('./2_Database/work_database_v1.db')

#### Extragem din baza de date interogarile dorite

In [4]:
df2 = pd.read_sql("select * from news where type = 'reliable'",db)

In [5]:
df1 = pd.read_sql("select * from news where type != 'reliable'",db)

#### Punem la un loc datele

In [7]:
df = pd.concat([df2,df1])

#### Facem update la index-ul din dataframe

In [29]:
df.reset_index(inplace = True)

In [34]:
df.drop(['level_0','index'],axis = 1,inplace = True)

In [41]:
df['index'] = df.index

In [43]:
df = df[['index', 'domain', 'type', 'content','title','authors','sentiment', 'magnitude']]

#### Facem un shuffle la dataframe

In [58]:
df = df.sample(frac=1)

#### Funtia de extragere sentiment cu Google API

In [98]:
def get_sentiment(df):
    for index, row in tqdm(df.iterrows()):
        text = df.loc[index].content
        document = types.Document(
             content=text,
            type=enums.Document.Type.PLAIN_TEXT,
            language = 'en')

        #Detects the sentiment of the text
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        df.at[index, 'sentiment'] = sentiment.score
        df.at[index, 'magnitude'] = sentiment.magnitude

In [113]:
get_sentiment(df)




0it [00:00, ?it/s]


1it [00:06,  6.49s/it]


2it [00:08,  5.03s/it]


3it [00:08,  3.77s/it]


4it [00:09,  2.71s/it]


5it [00:09,  1.97s/it]


6it [00:09,  1.53s/it]


7it [00:11,  1.47s/it]


8it [00:12,  1.42s/it]


9it [00:14,  1.56s/it]


KeyboardInterrupt: 

In [109]:
df.magnitude = df.magnitude.astype(float)

In [118]:
sample = df.sample(frac = 0.05)

In [124]:
sample.groupby('type').count()

,index,domain,content,title,authors,sentiment,magnitude
type,,,,,,,
conspiracy,41598,41598,41598,41598,41598,41598,41598
fake,44834,44834,44834,44834,44834,44834,44834
reliable,95547,95547,95547,95547,95547,95547,95547
unreliable,14920,14920,14920,14920,14920,14920,14920


In [ ]:
sample.to_csv("./sample.csv")